In [1]:
import os
import orjson

from models import multiscale_model as multiscale
from models import model
from models import track_model
from src.utils import visualize_detection_video, visualize_tracking_video

In [2]:
# videos_folder = "data/test_data/det_track/RGB/videos"
videos_folder = "data/Validation_Videos/IR"
model_path = "checkpoints/IR/yolov8n/best.pt"
output_folder = "outputs/eval/IR"

In [3]:
videos_path = [
    os.path.join(videos_folder, video)
    for video in os.listdir(videos_folder)
    if video.endswith(('.mp4', '.avi', '.mov'))
]

In [4]:
videos_path

['data/Validation_Videos/IR\\IR_BIRD_03897.mp4',
 'data/Validation_Videos/IR\\IR_BIRD_03998.mp4',
 'data/Validation_Videos/IR\\IR_BIRD_04099.mp4',
 'data/Validation_Videos/IR\\IR_BIRD_041100.mp4',
 'data/Validation_Videos/IR\\IR_BIRD_042101.mp4',
 'data/Validation_Videos/IR\\IR_BIRD_044103.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_082192.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_083193.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_085195.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_086196.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_087197.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_088198.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_089199.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_090200.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_091201.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_093203.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_094204.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_095205.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_096206.mp4',
 'data/Validation_Videos/IR\\IR_DRONE_09

In [5]:
multiscale_model = multiscale.DetectionModel(
                model_path=model_path,
                device="cuda",
            )
yolo_model = model.DetectionModel(
                model_path=model_path,
                device="cuda",
            )
# track_model = track_model.TrackingModel(
#                 multiscale_model
#             )

In [6]:
def save_log(
            detections,
            video_name,
            conf_threshold,
            iou_threshold,
            output_path
        ):
    new_detections = detections.copy()
    for frame_id, frame in enumerate(new_detections):
        for idx, (bbox, score, label) in enumerate(zip(frame["boxes"], frame["scores"], frame["labels"])):
            new_detections[frame_id]["boxes"][idx] = [float(coord) for coord in bbox]
            new_detections[frame_id]["scores"][idx] = float(score)
            new_detections[frame_id]["labels"][idx] = int(label)
            
    output = {
        "video_name": video_name,
        "conf_threshold": conf_threshold,
        "iou_threshold": iou_threshold,
        "frames": detections,
    }
    with open(output_path, "wb") as f:
        f.write(orjson.dumps(output, option=orjson.OPT_INDENT_2))
    print(f"Saved log to {output_path}")
    return output_path

In [ ]:
for iou in [0.1]:
    for conf in [0.1]:
        print(f"Running with iou: {iou}, conf: {conf}")
        for path in videos_path:
            video_name = os.path.splitext(os.path.basename(path))[0]
            print(f"Processing video: {video_name}")
            output_path = f"{output_folder}/iou_{iou}_conf_{conf}"
            
            yolo_det = yolo_model.video_detect(
                                            path,
                                            conf_threshold=conf,
                                            iou_threshold=iou
                                        )
            visualize_detection_video(
                video_path=path,
                detection_frames=yolo_det,
                output_path=f"{output_path}/{video_name}-yolo.mp4"
            )
            save_log(
                detections=yolo_det,
                video_name=video_name,
                conf_threshold=conf,
                iou_threshold=iou,
                output_path=f"{output_path}/{video_name}-yolo_det.json",
            )
            
            multiscale_det = multiscale_model.video_detect(
                                                    path,
                                                    conf_threshold=conf,
                                                    iou_threshold=iou    
                                                )    
            visualize_detection_video(
                video_path=path,
                detection_frames=multiscale_det,
                output_path=f"{output_path}/{video_name}.mp4"
            )
            save_log(
                detections=multiscale_det,
                video_name=video_name,
                conf_threshold=conf,
                iou_threshold=iou,
                output_path=f"{output_path}/{video_name}.json",
            )

Running with iou: 0.1, conf: 0.1
Processing video: IR_BIRD_03897


In [ ]:
# yolo_model = model.DetectionModel(
#                             model_path,
#                             conf_threshold=0.3,
#                             iou_threshold=0.45
#                             )
# multiscale_model = multiscale.DetectionModel(
#                                     model_path,
#                                     conf_threshold=0.3,
#                                     iou_threshold=0.1
#                                     )
# yolo_track_model = track_model.TrackingModel(
#                             yolo_model,
#                             )
# multiscale_track_model = track_model.TrackingModel(
#                             multiscale_model,
#                             )

In [ ]:
# for path in videos_path:
#     video_name = os.path.basename(path).removesuffix('.mp4').removesuffix('.avi').removesuffix('.mov')
#     print(f"Processing video: {video_name}")
    
#     # # Detection
#     # yolo_det = yolo_model.video_detect(path)
#     multiscale_det = multiscale_model.video_detect(path)
    
#     # visualize_detection_video(
#     #     video_path=path,
#     #     detection_frames=yolo_det,
#     #     output_path=f"{output_folder}/detection/{video_name}/yolo.mp4"
#     # )
#     visualize_detection_video(
#         video_path=path,
#         detection_frames=multiscale_det,
#         output_path=f"{output_folder}/detection/{video_name}.mp4"
#     )
    
#     # # Tracking
#     # yolo_track = yolo_track_model.video_track(path)
#     # multiscale_track = multiscale_track_model.video_track(path)
#     # visualize_tracking_video(
#     #     video_path=path,
#     #     tracking_frames=yolo_track,
#     #     output_path=f"{output_folder}/tracking/{video_name}/yolo.mp4"
#     # )
#     # visualize_tracking_video(
#     #     video_path=path,
#     #     tracking_frames=multiscale_track,
#     #     output_path=f"{output_folder}/tracking/{video_name}/multiscale.mp4"
#     # )